In [23]:
# pip install datasets[audio]
# !pip install datasets[vision]
# !pip install evaluate rouge_score
# !pip install bs4


In [24]:
# !pip install transformers
# !pip install torch
# !pip install accelerate -U
# pip install datasets

#try web hosting put the website on server for user testing
#train bert sentence by sentence
#check the ratio of + and - sentences, check if imbalanced, add if necessary


In [25]:
import torch
import pandas as pd
import numpy as np
from transformers import DataCollatorForSeq2Seq
# If there's a GPU available...
torch.cuda.is_available()   

True

In [26]:
from datasets import load_dataset
from bs4 import BeautifulSoup
import requests

# spanish_dataset = load_dataset("amazon_reviews_multi", "es")
# english_dataset = load_dataset("amazon_reviews_multi", "en")
# english_dataset

# billsum = load_dataset("billsum", split="ca_test")
# df = pd.read_parquet('models/train-00000-of-00003.parquet')
cnn_dailymail = load_dataset("cnn_dailymail", '3.0.0')

In [27]:

cnn_dailymail

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [28]:
cnn_dailymail["train"][0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [29]:
# model training code reference https://huggingface.co/docs/transformers/tasks/summarization

from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [30]:
prefix = "summarize: "

def preprocess_function(data):
    inputs = [prefix + doc for doc in data["article"]] 
    outputs = data["highlights"]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=outputs, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [31]:
tokenized_cnn_dailymail = cnn_dailymail.map(preprocess_function, batched=True)

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [32]:


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [33]:
print(cnn_dailymail["train"].num_rows)
print(cnn_dailymail["train"][0])

287113
{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box offi

In [36]:

# rogue scorer reference https://pypi.org/project/rouge-score/
# training code referenced from https://huggingface.co/docs/transformers/tasks/summarization
from datasets import load_metric
#train the model
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
rouge = load_metric("rouge")
# Define trainer



def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, 
                                            skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_predictions,
                            references=decoded_labels, 
                            rouge_types=["rouge1", "rouge2", "rougeL"])
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-cnn-dailymail-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

class T5Trainer(Seq2SeqTrainer):
    def _compute_metrics(self, eval_pred):
        return compute_metrics(eval_pred)

trainer = T5Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_cnn_dailymail["train"],
    eval_dataset=tokenized_cnn_dailymail["validation"],
    compute_metrics=compute_metrics,

)
    






c:\Users\zacha\anaconda3\lib\site-packages\datasets\load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [37]:



trainer.train()
# trainer.evaluate()


  0%|          | 0/17945 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.save_model("summodels")
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)
#aas of 
#{'eval_loss': 2.4772226810455322, 'eval_rouge1': 0.1454,
#  'eval_rouge2': 0.0554, 'eval_rougeL': 0.1213, 'eval_rougeLsum': 0.1216,
#  'eval_gen_len': 19.0, 'eval_runtime': 218.4295, 'eval_samples_per_second': 1.135, 'eval_steps_per_second': 0.073, 'epoch': 4.0}

c:\Users\zacha\anaconda3\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/16 [00:00<?, ?it/s]

{'eval_loss': 2.4772226810455322, 'eval_rouge1': 0.1454, 'eval_rouge2': 0.0554, 'eval_rougeL': 0.1213, 'eval_rougeLsum': 0.1216, 'eval_gen_len': 19.0, 'eval_runtime': 218.4295, 'eval_samples_per_second': 1.135, 'eval_steps_per_second': 0.073, 'epoch': 4.0}


In [ ]:
# import evaluate

# rouge_score = evaluate.load("rouge")

In [ ]:

# list of sample urls for input
# URL="https://www.snopes.com/fact-check/strikes-on-yemen-image/"
# URL='https://www.dagens.com/news/russia-outraged-after-major-attack'
# URL='https://www.snopes.com/fact-check/toyota-tundra-giveaway/'
URL='https://content.time.com/time/specials/packages/article/0,28804,1860871_1860876_1861013,00.html'
# no webscraping, sipmle text

In [39]:
from transformers import pipeline

summarizer = pipeline("summarization", model="t5-cnn-dailymail-model/checkpoint-17500")


OSError: Can't load tokenizer for 't5-cnn-dailymail-model/checkpoint-17500'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 't5-cnn-dailymail-model/checkpoint-17500' is the correct path to a directory containing all relevant files for a T5TokenizerFast tokenizer.

In [ ]:
# reference from https://github.com/nicknochnack/Longform-Summarization-with-Hugging-Face/blob/main/LongSummarization.ipynb
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

ConnectionError: HTTPSConnectionPool(host='content.time.com', port=443): Max retries exceeded with url: /time/specials/packages/article/0,28804,1860871_1860876_1861013,00.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000014E6AC27A90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
ARTICLE
max_chunk = 500
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])


0


In [ ]:
res = summarizer(chunks, max_length=300, min_length=50, do_sample=False)
# print(summarizer(text))
' '.join([summ['summary_text'] for summ in res])
text = ' '.join([summ['summary_text'] for summ in res])
print (text)


Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors


TIME looks at 10 of the world's most enduring conspiracy theories 2 of 10 Not since the JFK assassination has there been a national tragedy so heavily imprinted in American minds — or that has given rise to quite as many alternative explanations . videos and photographs of the two planes striking the World Trade Center towers are famous around the world, the sheer profusion of documentary evidence has only provided even more fodder for conspiracy theories .
